In [78]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
    
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

chain.predict(question="My name is tom")

'Hello Tom! How can I assist you today?'

In [79]:
chain.predict(question="i live in seoul")


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [80]:
chain.predict(question="What is my Name?")

"I'm sorry, I do not have the ability to know your name."